In [1]:
!pip install snowflake-connector-python snowflake-ingest

In [16]:
import snowflake.connector
import random
import json
import os
from decimal import Decimal

# Connect to Snowflake
conn = snowflake.connector.connect(
    user='bigdatadeiyo',
    password='Ramikasatha1130',
    account='FNMMMLV-XE53969',
    warehouse='COMPUTE_WH',
    database='TEST1',
    schema='STREAM'
)

cursor = conn.cursor()

def get_existing_patient_ids():
    """Fetch existing PATIENTID values from the Snowflake table."""
    cursor.execute("SELECT PATIENTID FROM TEST1")
    return {row[0] for row in cursor.fetchall()}

def generate_decimal(min_val, max_val, precision=15):
    """Generate a decimal number with the specified precision."""
    return round(random.uniform(min_val, max_val), precision)

def generate_patient_data(num_records=50):
    existing_ids = get_existing_patient_ids()
    records = []
    doctor_name = "DrXXXConfid"
    
    while len(records) < num_records:
        patient_id = random.randint(1000, 3000) if random.choice([True, False]) else random.randint(5200, 9999)
        if patient_id in existing_ids:
            continue  # Avoid duplicate patient IDs
        
        record = {
            'PATIENTID': patient_id,
            'AGE': random.randint(50, 90),
            'GENDER': random.randint(0, 1),
            'ETHNICITY': random.randint(0, 3),
            'EDUCATIONLEVEL': random.randint(0, 3),
            'BMI': generate_decimal(15, 40),
            'SMOKING': random.randint(0, 1),
            'ALCOHOLCONSUMPTION': generate_decimal(0, 20, 19),
            'PHYSICALACTIVITY': generate_decimal(0, 10, 18),
            'DIETQUALITY': generate_decimal(0, 10, 18),
            'SLEEPQUALITY': generate_decimal(4, 10, 16),
            'FAMILYHISTORYPARKINSONS': random.randint(0, 1),
            'TRAUMATICBRAININJURY': random.randint(0, 1),
            'HYPERTENSION': random.randint(0, 1),
            'DIABETES': random.randint(0, 1),
            'DEPRESSION': random.randint(0, 1),
            'STROKE': random.randint(0, 1),
            'SYSTOLICBP': random.randint(90, 180),
            'DIASTOLICBP': random.randint(60, 120),
            'CHOLESTEROLTOTAL': generate_decimal(150, 300, 14),
            'CHOLESTEROLLDL': generate_decimal(50, 200, 15),
            'CHOLESTEROLHDL': generate_decimal(20, 100, 15),
            'CHOLESTEROLTRIGLYCERIDES': generate_decimal(50, 400, 15),
            'UPDRS': generate_decimal(0, 199, 18),
            'MOCA': generate_decimal(0, 30, 18),
            'FUNCTIONALASSESSMENT': generate_decimal(0, 10, 18),
            'TREMOR': random.randint(0, 1),
            'RIGIDITY': random.randint(0, 1),
            'BRADYKINESIA': random.randint(0, 1),
            'POSTURALINSTABILITY': random.randint(0, 1),
            'SPEECHPROBLEMS': random.randint(0, 1),
            'SLEEPDISORDERS': random.randint(0, 1),
            'CONSTIPATION': random.randint(0, 1),
            'DIAGNOSIS': random.randint(0, 1),
            'DOCTORINCHARGE': doctor_name
        }
        records.append(record)
    
    return records

try:
    records = generate_patient_data(50)
    json_file = "patient_data.json"
    
    with open(json_file, "w") as f:
        for record in records:
            json.dump(record, f)
            f.write("\n")
    
    cursor.execute("CREATE TEMPORARY STAGE IF NOT EXISTS temp_stage")
    cursor.execute(f"PUT file://{json_file} @temp_stage")
    
    cursor.execute("""
    COPY INTO TEST1
    FROM @temp_stage/patient_data.json.gz
    FILE_FORMAT = (TYPE = 'JSON')
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    """)
    
    print(f"Successfully loaded {len(records)} records into TEST1")
except Exception as e:
    print(f"Error loading data: {e}")
finally:
    try:
        cursor.execute("DROP STAGE IF EXISTS temp_stage")
        os.remove(json_file)
    except Exception as e:
        print(f"Error during cleanup: {e}")
    cursor.close()
    conn.close()
    print("Connections closed")

Successfully loaded 50 records into TEST1
Connections closed
